<a href="https://colab.research.google.com/github/nklbigone/DIC-ML/blob/First-Assignment/Spriont_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**[Problem 1] Cross Validation**

In [5]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score

# loading the csv of the dataset
df = pd.read_csv('/application_train.csv')
df = df.select_dtypes('number')

# cleaning the dataset by filling the empy data(null)
cleaned_df = df.fillna(0)

# get only existing data with no missing values
cleaned_df = cleaned_df[cleaned_df.columns[~cleaned_df.isnull().all()]]

# separating them into variables
y = cleaned_df['TARGET']
X = cleaned_df.drop(['TARGET'], axis=1)

X = X.to_numpy()

kf = KFold(n_splits=2)

for train_index, test_index in kf.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

TRAIN: [15541 15542 15543 ... 31078 31079 31080] TEST: [    0     1     2 ... 15538 15539 15540]
TRAIN: [    0     1     2 ... 15538 15539 15540] TEST: [15541 15542 15543 ... 31078 31079 31080]


In [6]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train_trans = scaler.transform(X_train)
X_test_trans = scaler.transform(X_test)


**[Problem 2] Grid search**

In [7]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

# checking which model and params are best
model_params = {
    'random_forest':{
        'model': RandomForestClassifier(),
        'params': {
            'n_estimators': [1,5,10]
        }
    },
    'logic_regression':{
        'model': LogisticRegression(solver="liblinear",multi_class="auto"),
        'params': {
            'C': [1,5,10]
        }
    }
}

# defining an array to store the scores
scores = []

for model_name,mp in model_params.items():
    clf = GridSearchCV(mp['model'],mp['params'], return_train_score=False)
    clf.fit(X_train_trans,y_train)
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })

best_model_params = pd.DataFrame(scores,columns=['model','best_score','best_params'])
best_model_params

,model,best_score,best_params
0,random_forest,0.851164,{'n_estimators': 10}
1,logic_regression,0.920983,{'C': 1}


**[Problem 3] Survey from Kaggle Notebooks**

Model Hyperparameter Optimization is the  points of choice or configuration that allow a machine learning model to be customized for a specific task or dataset.

Parameters are different from hyperparameters. Parameters are learned automatically; hyperparameters are set manually to help guide the learning process.

the Gradient Boosting Algorithm for Machine Learning The origin of boosting from learning theory and AdaBoost.
How gradient boosting works including the loss function, weak learners and the additive model.
How to improve performance over the base algorithm with various regularization schemes.
Modal explainability 
Early Stopping to Avoid Overtraining Neural Networks A compromise is to train on the training dataset but to stop training at the point when performance on a validation dataset starts to degrade. This simple, effective, and widely used approach to training neural networks is called early stopping.

**[Problem 4] Creating a model with high generalization performance**

In [8]:
import lightgbm as lgb

# creating an instance of the model
model = lgb.LGBMClassifier()

# save the default params
default_params = model.get_params()

# number of folds
N_FOLDS = 5

# creating a dataset
train_set = lgb.Dataset(data = X_train)

# Cross validation results when avoid overfitting
cv_results = lgb.cv(default_params, train_set, num_boost_round = 10000, early_stopping_rounds = 100, metrics = 'auc', nfold = N_FOLDS, seed = 42)

# displaying the results
print('The maximum validation ROC AUC was: {:.5f}.'.format(cv_results['auc-mean'][-1]))
print('The optimal number of boosting rounds (estimators) was {}.'.format(len(cv_results['auc-mean'])))

/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:430: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.7/dist-packages/lightgbm/basic.py:741: UserWarning: silent keyword has been found in `params` and will be ignored.
Please use silent argument of the Dataset constructor to pass this parameter.
  .format(key))


The maximum validation ROC AUC was: 1.00000.
The optimal number of boosting rounds (estimators) was 1.


imported the whole dataset
created a subset of only numbers
split the data using kfold
used gridsearchCV to find the best model and params to fine tune my classiffiers

**[Problem 5] Final model selection**

In [16]:

# loading the csv of the test dataset
test_df = pd.read_csv('/application_test.csv')

# cleaning the dataset by removing the empy data(null)
test_cleaned_df = test_df.fillna(0)

# separating them into variables
test_X = test_cleaned_df.select_dtypes('number')

# standardizing the data
test_scaler = StandardScaler()
test_X_test_trans = scaler.fit_transform(test_X)

# predicting
test_reg_pred = clf.predict(test_X_test_trans)

kgl_submission = pd.concat([test_df['SK_ID_CURR'], pd.Series(test_reg_pred, name='TARGET')], axis=1)
kgl_submission.to_csv('application_test1.csv ', index=False)